In [19]:
import pandas as pd 
import numpy as np 
from bs4 import BeautifulSoup
from requests import get 

In [20]:
!pip install requests 
!pip install beautifulsoup4

In [21]:
def get_resp(base_url1, player_name,base_url2):
    url = base_url1+player+base_url2
    response = get(url)
    print(response.status_code)
    return(response)

def parser(response, cols):
    nfl = BeautifulSoup(response.content, 'html.parser')
    table = nfl.find(lambda tag: tag.name=='table') 
    rows = table.find_all('tr')
    df = pd.DataFrame(columns = cols)
    
    for i in range(1, len(rows)):
        print(i)
        a = rows[i].get_text()
        b = a.replace('\n', ',')
        b = b.strip()
        b = b.replace(' ','')
        c = b[1:].split(',')
        d = []
        for j in range(0, len(c)): 
            if c[j]!='': 
                d.append(c[j])
        for j in range(0,len(cols)):

            try:
                #print(d[j])
                df.loc[i-1, cols[j]] = d[j]
            except: 
                df.loc[i-1, cols[j]] = ''
            
            
    return(df)

def get_def_stats(url, cols):
    tables = pd.read_html(url)
    df = tables[1]
    df.columns = cols
    #df.columns = ['Week','Day', 'Date','Time','Link','W/L','OT','Rec','H/A','Opp','TM','Opp Score','Offense 1st','Off tot yd', 'off pass yds',
             #'Off rush Yards', 'Off TO', 'Def 1st D', 'Def Tot Yd','Deff Pass Yd', 'Def Rush Yards', 'Def TO','Ex Off','Exp Def','Ex Sp. Tms' ]
    
    return(df)

In [22]:
base1 = 'https://www.nfl.com/players/'
player = 'josh-allen-4'
base2 = '/stats/logs/'
cols = ['Wk', 'Date', 'Opp', 'Res','Comp', 'Att','Yds','Avg','TD','Int','SCK','Scky','Rate','Att1','Yds1','Avg1','Td1','Fum','Lost']
    
resp = get_resp(base1, player, base2)
play_stats = parser(resp, cols)
play_stats

200
1
2
3
4
5
6
7
8
9
10


,Wk,Date,Opp,Res,Comp,Att,Yds,Avg,TD,Int,SCK,Scky,Rate,Att1,Yds1,Avg1,Td1,Fum,Lost
0,10,11/15/2020,@Cardinals,L30-32,32,49,284,5.8,2,2,0,0,77.3,7,38,5.4,0,,
1,9,11/08/2020,Seahawks,W44-34,31,38,415,10.9,3,0,7,29,138.5,7,14,2,1,,
2,8,11/01/2020,Patriots,W24-21,11,18,154,8.6,0,1,1,5,65.5,10,23,2.3,1,,
3,3,09/27/2020,Rams,W35-32,24,33,311,9.4,4,1,4,37,128.9,4,8,2,1,1,1
4,7,10/25/2020,@Jets,W18-10,30,43,307,7.1,0,0,2,11,90,11,61,5.5,0,1,1
5,2,09/20/2020,@Dolphins,W31-28,24,35,415,11.9,4,0,1,4,146.7,4,19,4.8,0,,
6,6,10/19/2020,Chiefs,L17-26,14,27,122,4.5,2,1,0,0,73.4,8,42,5.3,0,,
7,1,09/13/2020,Jets,W27-17,33,46,312,6.8,2,0,3,6,104.6,14,57,4.1,1,2,2
8,5,10/13/2020,@Titans,L16-42,26,41,263,6.4,2,2,1,10,77.6,4,18,4.5,0,,
9,4,10/04/2020,@Raiders,W30-23,24,34,288,8.5,2,0,1,14,115.8,3,-1,-0.3,1,,


In [23]:
cols_to_numeric = ['Wk','Comp','Att','Yds','Avg','TD','Int','SCK','Scky','Rate','Att1','Yds1','Avg1','Td1','Fum','Lost']
for i in range(0, len(cols_to_numeric)): 
    #print(i)
    col = cols_to_numeric[i]
    #print(col)
    play_stats[col] = pd.to_numeric(play_stats[col])
    
play_stats

,Wk,Date,Opp,Res,Comp,Att,Yds,Avg,TD,Int,SCK,Scky,Rate,Att1,Yds1,Avg1,Td1,Fum,Lost
0,10,11/15/2020,@Cardinals,L30-32,32,49,284,5.8,2,2,0,0,77.3,7,38,5.4,0,NaN,NaN
1,9,11/08/2020,Seahawks,W44-34,31,38,415,10.9,3,0,7,29,138.5,7,14,2.0,1,NaN,NaN
2,8,11/01/2020,Patriots,W24-21,11,18,154,8.6,0,1,1,5,65.5,10,23,2.3,1,NaN,NaN
3,3,09/27/2020,Rams,W35-32,24,33,311,9.4,4,1,4,37,128.9,4,8,2.0,1,1.0,1.0
4,7,10/25/2020,@Jets,W18-10,30,43,307,7.1,0,0,2,11,90.0,11,61,5.5,0,1.0,1.0
5,2,09/20/2020,@Dolphins,W31-28,24,35,415,11.9,4,0,1,4,146.7,4,19,4.8,0,NaN,NaN
6,6,10/19/2020,Chiefs,L17-26,14,27,122,4.5,2,1,0,0,73.4,8,42,5.3,0,NaN,NaN
7,1,09/13/2020,Jets,W27-17,33,46,312,6.8,2,0,3,6,104.6,14,57,4.1,1,2.0,2.0
8,5,10/13/2020,@Titans,L16-42,26,41,263,6.4,2,2,1,10,77.6,4,18,4.5,0,NaN,NaN
9,4,10/04/2020,@Raiders,W30-23,24,34,288,8.5,2,0,1,14,115.8,3,-1,-0.3,1,NaN,NaN


In [24]:
play_stats.sort_values(by = 'Wk', inplace = True)
play_stats.reset_index(inplace = True, drop = True)
play_stats

,Wk,Date,Opp,Res,Comp,Att,Yds,Avg,TD,Int,SCK,Scky,Rate,Att1,Yds1,Avg1,Td1,Fum,Lost
0,1,09/13/2020,Jets,W27-17,33,46,312,6.8,2,0,3,6,104.6,14,57,4.1,1,2.0,2.0
1,2,09/20/2020,@Dolphins,W31-28,24,35,415,11.9,4,0,1,4,146.7,4,19,4.8,0,NaN,NaN
2,3,09/27/2020,Rams,W35-32,24,33,311,9.4,4,1,4,37,128.9,4,8,2.0,1,1.0,1.0
3,4,10/04/2020,@Raiders,W30-23,24,34,288,8.5,2,0,1,14,115.8,3,-1,-0.3,1,NaN,NaN
4,5,10/13/2020,@Titans,L16-42,26,41,263,6.4,2,2,1,10,77.6,4,18,4.5,0,NaN,NaN
5,6,10/19/2020,Chiefs,L17-26,14,27,122,4.5,2,1,0,0,73.4,8,42,5.3,0,NaN,NaN
6,7,10/25/2020,@Jets,W18-10,30,43,307,7.1,0,0,2,11,90.0,11,61,5.5,0,1.0,1.0
7,8,11/01/2020,Patriots,W24-21,11,18,154,8.6,0,1,1,5,65.5,10,23,2.3,1,NaN,NaN
8,9,11/08/2020,Seahawks,W44-34,31,38,415,10.9,3,0,7,29,138.5,7,14,2.0,1,NaN,NaN
9,10,11/15/2020,@Cardinals,L30-32,32,49,284,5.8,2,2,0,0,77.3,7,38,5.4,0,NaN,NaN


In [25]:
play_stats.dtypes

Wk        int64
Date     object
Opp      object
Res      object
Comp      int64
Att       int64
Yds       int64
Avg     float64
TD        int64
Int       int64
SCK       int64
Scky      int64
Rate    float64
Att1      int64
Yds1      int64
Avg1    float64
Td1       int64
Fum     float64
Lost    float64
dtype: object

In [26]:


def_df = get_def_stats(r'https://www.pro-football-reference.com/teams/mia/2020.htm', ['Week','Day', 'Date','Time','Link','W/L','OT','Rec','H/A','Opp','TM','Opp Score','Offense 1st','Off tot yd', 'off pass yds',
             'Off rush Yards', 'Off TO', 'Def 1st D', 'Def Tot Yd','Deff Pass Yd', 'Def Rush Yards', 'Def TO','Ex Off','Exp Def','Ex Sp. Tms' ])
def_df

,Week,Day,Date,Time,Link,W/L,OT,Rec,H/A,Opp,...,Off rush Yards,Off TO,Def 1st D,Def Tot Yd,Deff Pass Yd,Def Rush Yards,Def TO,Ex Off,Exp Def,Ex Sp. Tms
0,1,Sun,September 13,1:00PM ET,boxscore,L,NaN,0-1,@,New England Patriots,...,87.0,3.0,29.0,357.0,140.0,217.0,1.0,-2.20,-14.11,5.64
1,2,Sun,September 20,1:00PM ET,boxscore,L,NaN,0-2,NaN,Buffalo Bills,...,99.0,NaN,23.0,523.0,411.0,112.0,1.0,12.70,-21.16,2.30
2,3,Thu,September 24,8:20PM ET,boxscore,W,NaN,1-2,@,Jacksonville Jaguars,...,138.0,NaN,22.0,318.0,246.0,72.0,2.0,12.93,5.61,-1.87
3,4,Sun,October 4,1:00PM ET,boxscore,L,NaN,1-3,NaN,Seattle Seahawks,...,103.0,2.0,26.0,441.0,343.0,98.0,1.0,7.12,-16.11,1.32
4,5,Sun,October 11,4:05PM ET,boxscore,W,NaN,2-3,@,San Francisco 49ers,...,94.0,NaN,19.0,259.0,128.0,131.0,3.0,19.36,9.48,0.03
5,6,Sun,October 18,4:05PM ET,boxscore,W,NaN,3-3,NaN,New York Jets,...,110.0,2.0,13.0,263.0,148.0,115.0,1.0,0.32,18.17,3.79
6,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bye Week,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,8,Sun,November 1,1:00PM ET,boxscore,W,NaN,4-3,NaN,Los Angeles Rams,...,55.0,2.0,31.0,471.0,340.0,131.0,4.0,-21.63,18.41,12.41
8,9,Sun,November 8,4:25PM ET,boxscore,W,NaN,5-3,@,Arizona Cardinals,...,91.0,NaN,26.0,442.0,264.0,178.0,1.0,15.28,-17.94,6.36
9,10,Sun,November 15,4:05PM ET,boxscore,W,NaN,6-3,NaN,Los Angeles Chargers,...,111.0,1.0,19.0,273.0,174.0,99.0,1.0,-7.96,1.88,14.67


In [27]:
def_df.columns.tolist()

['Week',
 'Day',
 'Date',
 'Time',
 'Link',
 'W/L',
 'OT',
 'Rec',
 'H/A',
 'Opp',
 'TM',
 'Opp Score',
 'Offense 1st',
 'Off tot yd',
 'off pass yds',
 'Off rush Yards',
 'Off TO',
 'Def 1st D',
 'Def Tot Yd',
 'Deff Pass Yd',
 'Def Rush Yards',
 'Def TO',
 'Ex Off',
 'Exp Def',
 'Ex Sp. Tms']

In [28]:
cols_to_keep = ['Week','Opp Score','Opp','Def Tot Yd','Deff Pass Yd','Def Rush Yards','Def TO']
df3 = def_df[cols_to_keep]
df3

,Week,Opp Score,Opp,Def Tot Yd,Deff Pass Yd,Def Rush Yards,Def TO
0,1,21.0,New England Patriots,357.0,140.0,217.0,1.0
1,2,31.0,Buffalo Bills,523.0,411.0,112.0,1.0
2,3,13.0,Jacksonville Jaguars,318.0,246.0,72.0,2.0
3,4,31.0,Seattle Seahawks,441.0,343.0,98.0,1.0
4,5,17.0,San Francisco 49ers,259.0,128.0,131.0,3.0
5,6,0.0,New York Jets,263.0,148.0,115.0,1.0
6,7,NaN,Bye Week,NaN,NaN,NaN,NaN
7,8,17.0,Los Angeles Rams,471.0,340.0,131.0,4.0
8,9,31.0,Arizona Cardinals,442.0,264.0,178.0,1.0
9,10,21.0,Los Angeles Chargers,273.0,174.0,99.0,1.0


In [29]:
cols_to_numeric = ['Week','Opp Score','Def Tot Yd','Deff Pass Yd','Def Rush Yards','Def TO']

for i in range(0, len(cols_to_numeric)): 
    col = cols_to_numeric[i]
    df3[col] = pd.to_numeric(df3[col])

df3.dtypes




<ipython-input-29-691390b6d0d1>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3[col] = pd.to_numeric(df3[col])


Week                int64
Opp Score         float64
Opp                object
Def Tot Yd        float64
Deff Pass Yd      float64
Def Rush Yards    float64
Def TO            float64
dtype: object

In [30]:
df3.sort_values(by = 'Week')
df3.reset_index(inplace = True, drop = True)
df3

,Week,Opp Score,Opp,Def Tot Yd,Deff Pass Yd,Def Rush Yards,Def TO
0,1,21.0,New England Patriots,357.0,140.0,217.0,1.0
1,2,31.0,Buffalo Bills,523.0,411.0,112.0,1.0
2,3,13.0,Jacksonville Jaguars,318.0,246.0,72.0,2.0
3,4,31.0,Seattle Seahawks,441.0,343.0,98.0,1.0
4,5,17.0,San Francisco 49ers,259.0,128.0,131.0,3.0
5,6,0.0,New York Jets,263.0,148.0,115.0,1.0
6,7,NaN,Bye Week,NaN,NaN,NaN,NaN
7,8,17.0,Los Angeles Rams,471.0,340.0,131.0,4.0
8,9,31.0,Arizona Cardinals,442.0,264.0,178.0,1.0
9,10,21.0,Los Angeles Chargers,273.0,174.0,99.0,1.0


In [31]:
#'Def Passing Yards Avg','Def Running Yards Avg','Turnovers Avg'
df3.dropna(inplace = True)
df3['Def Passing Yards Avg'] = df3['Deff Pass Yd'].rolling(5).mean()
df3['Def Running Yards Avg'] = df3['Def Rush Yards'].rolling(5).mean()
df3['Turnovers Avg'] = df3['Def TO'].rolling(5).mean()
df3

<ipython-input-31-1000af4aa8ec>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3.dropna(inplace = True)
<ipython-input-31-1000af4aa8ec>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['Def Passing Yards Avg'] = df3['Deff Pass Yd'].rolling(5).mean()
<ipython-input-31-1000af4aa8ec>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ve

,Week,Opp Score,Opp,Def Tot Yd,Deff Pass Yd,Def Rush Yards,Def TO,Def Passing Yards Avg,Def Running Yards Avg,Turnovers Avg
0,1,21.0,New England Patriots,357.0,140.0,217.0,1.0,NaN,NaN,NaN
1,2,31.0,Buffalo Bills,523.0,411.0,112.0,1.0,NaN,NaN,NaN
2,3,13.0,Jacksonville Jaguars,318.0,246.0,72.0,2.0,NaN,NaN,NaN
3,4,31.0,Seattle Seahawks,441.0,343.0,98.0,1.0,NaN,NaN,NaN
4,5,17.0,San Francisco 49ers,259.0,128.0,131.0,3.0,253.6,126.0,1.6
5,6,0.0,New York Jets,263.0,148.0,115.0,1.0,255.2,105.6,1.6
7,8,17.0,Los Angeles Rams,471.0,340.0,131.0,4.0,241.0,109.4,2.2
8,9,31.0,Arizona Cardinals,442.0,264.0,178.0,1.0,244.6,130.6,2.0
9,10,21.0,Los Angeles Chargers,273.0,174.0,99.0,1.0,210.8,130.8,2.0
10,11,20.0,Denver Broncos,459.0,270.0,189.0,2.0,239.2,142.4,1.8


In [32]:
play_stats
play_stats.fillna(0, inplace = True)
play_stats

,Wk,Date,Opp,Res,Comp,Att,Yds,Avg,TD,Int,SCK,Scky,Rate,Att1,Yds1,Avg1,Td1,Fum,Lost
0,1,09/13/2020,Jets,W27-17,33,46,312,6.8,2,0,3,6,104.6,14,57,4.1,1,2.0,2.0
1,2,09/20/2020,@Dolphins,W31-28,24,35,415,11.9,4,0,1,4,146.7,4,19,4.8,0,0.0,0.0
2,3,09/27/2020,Rams,W35-32,24,33,311,9.4,4,1,4,37,128.9,4,8,2.0,1,1.0,1.0
3,4,10/04/2020,@Raiders,W30-23,24,34,288,8.5,2,0,1,14,115.8,3,-1,-0.3,1,0.0,0.0
4,5,10/13/2020,@Titans,L16-42,26,41,263,6.4,2,2,1,10,77.6,4,18,4.5,0,0.0,0.0
5,6,10/19/2020,Chiefs,L17-26,14,27,122,4.5,2,1,0,0,73.4,8,42,5.3,0,0.0,0.0
6,7,10/25/2020,@Jets,W18-10,30,43,307,7.1,0,0,2,11,90.0,11,61,5.5,0,1.0,1.0
7,8,11/01/2020,Patriots,W24-21,11,18,154,8.6,0,1,1,5,65.5,10,23,2.3,1,0.0,0.0
8,9,11/08/2020,Seahawks,W44-34,31,38,415,10.9,3,0,7,29,138.5,7,14,2.0,1,0.0,0.0
9,10,11/15/2020,@Cardinals,L30-32,32,49,284,5.8,2,2,0,0,77.3,7,38,5.4,0,0.0,0.0


In [33]:
x_cols = ['passing_att','passing_ints_pts', 'passing_tds_pts', 'passing_twoptm_pts', 'passing_yds_pts', 
          'rushing_td_pts','rushing_yds_pts','fumbles_lost_pts','Def Passing Yards Avg',
          'Def Running Yards Avg','Turnovers Avg']

week = 10 

df_for_regression = pd.DataFrame()
df_for_regression['Week'] = play_stats['Wk']
df_for_regression['passing_att'] = play_stats['Att']
df_for_regression['passing_ints_pts'] = play_stats['Int'] * -1 
df_for_regression['passing_tds_pts'] = play_stats['TD']*4
#df_for_regression['passing_twoptm_pts'] = play_stats['']
df_for_regression['passing_yds_pts'] = play_stats['Yds']*0.04
df_for_regression['rushing_td_pts'] = play_stats['Td1'] * 6
df_for_regression['fumbles_lost_pts'] = play_stats['Fum'] * -1
df_for_regression['rushing_yds_pts'] = play_stats['Yds1'] * 0.1 
df_for_regression

,Week,passing_att,passing_ints_pts,passing_tds_pts,passing_yds_pts,rushing_td_pts,fumbles_lost_pts,rushing_yds_pts
0,1,46,0,8,12.48,6,-2.0,5.7
1,2,35,0,16,16.60,0,-0.0,1.9
2,3,33,-1,16,12.44,6,-1.0,0.8
3,4,34,0,8,11.52,6,-0.0,-0.1
4,5,41,-2,8,10.52,0,-0.0,1.8
5,6,27,-1,8,4.88,0,-0.0,4.2
6,7,43,0,0,12.28,0,-1.0,6.1
7,8,18,-1,0,6.16,6,-0.0,2.3
8,9,38,0,12,16.60,6,-0.0,1.4
9,10,49,-2,8,11.36,0,-0.0,3.8


In [34]:
week_for_off = 10 

x_cols = ['passing_att','passing_ints_pts', 'passing_tds_pts', 'passing_yds_pts', 
          'rushing_td_pts','rushing_yds_pts','fumbles_lost_pts','Def Passing Yards Avg',
          'Def Running Yards Avg','Turnovers Avg']

coefs = [-2.3006,-2.5185,1.07,5.2943,-2.3527,-1.4089,0.1970,-0.0354,-5.6371]
cols_player = df_for_regression.columns.tolist()[1:]
pts = 0
mask = df_for_regression['Week'] == week_for_off
df_for_calcs = df_for_regression[mask]
df_for_calcs
for i in range(0, len(cols_player)):
    pts = pts + df_for_calcs[x_cols[i]]*coefs[i]

week_for_def = 10 
mask = df3['Week'] == week_for_def
def_df_for_calcs = df3[mask]
def_cols = ['Def Passing Yards Avg','Def Running Yards Avg','Turnovers Avg']
for i in range(0, len(def_cols)): 
    pts = pts + coefs[i+6]*def_df_for_calcs[def_cols[i]]
pts    

9   -18.719892
dtype: float64

In [35]:
df3

,Week,Opp Score,Opp,Def Tot Yd,Deff Pass Yd,Def Rush Yards,Def TO,Def Passing Yards Avg,Def Running Yards Avg,Turnovers Avg
0,1,21.0,New England Patriots,357.0,140.0,217.0,1.0,NaN,NaN,NaN
1,2,31.0,Buffalo Bills,523.0,411.0,112.0,1.0,NaN,NaN,NaN
2,3,13.0,Jacksonville Jaguars,318.0,246.0,72.0,2.0,NaN,NaN,NaN
3,4,31.0,Seattle Seahawks,441.0,343.0,98.0,1.0,NaN,NaN,NaN
4,5,17.0,San Francisco 49ers,259.0,128.0,131.0,3.0,253.6,126.0,1.6
5,6,0.0,New York Jets,263.0,148.0,115.0,1.0,255.2,105.6,1.6
7,8,17.0,Los Angeles Rams,471.0,340.0,131.0,4.0,241.0,109.4,2.2
8,9,31.0,Arizona Cardinals,442.0,264.0,178.0,1.0,244.6,130.6,2.0
9,10,21.0,Los Angeles Chargers,273.0,174.0,99.0,1.0,210.8,130.8,2.0
10,11,20.0,Denver Broncos,459.0,270.0,189.0,2.0,239.2,142.4,1.8


In [36]:
6,7,8

(6, 7, 8)